# Implementación de un ChatBot con memoria y resumen de la conversación

En esta práctica se implementa un ChatBot con memoria, haciendo uso de Gradio para la visualización de una interfaz gráfica, y de la API de OpenAI para responder las diferentes preguntas o cuestiones del usuario. 

Para implementar la memoria, se concatenan en un vector las preguntas y respuestas del usuario y del ChatGPT, de tal forma que con cada nueva pregunta del usuario, se incorpora también toda la conversación. Además de esto, se realiza un resumen de la conversación para evitar superar el límite de tokens de entrada. 

Resuelto por: Javier Navarro Lázaro

# Instalación de librerías

In [ ]:
!pip install openai==0.28
!pip install python-dotenv
!pip install gradio

In [ ]:
import gradio as gr
import openai
from dotenv import load_dotenv
import os

load_dotenv()

# Configurar la API de Azure OpenAI
openai.api_type = os.getenv("AZURE_OPENAI_API_TYPE")
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# Creación de un chatbot con memoria y resumen

In [ ]:
# Función para manejar las solicitudes del chatbot sin memoria
def chatbot_no_memory(message):
    # El mensaje se envía sin incluir historial previo
    messages = [{"role": "system", "content": "Eres un asistente útil y amigable."}]

    # Añadir el nuevo mensaje del usuario
    messages.append({"role": "user", "content": message})

    # Solicitud a OpenAI para generar la respuesta
    response = openai.ChatCompletion.create(
        engine="gpt-4-turbo",  # Nombre del deployment
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )

    # Obtener la respuesta del asistente
    bot_response = response['choices'][0]['message']['content']

    # Devolver solo la respuesta del bot (sin historial)
    return bot_response

# Función para manejar las solicitudes del chatbot sin memoria
def chatbot_memory_full(message, chat_history):
    # El mensaje se envía sin incluir historial previo

    previous_msgs_str = ""
    for msg in chat_history:
      previous_msgs_str += msg[0]
    messages = [{"role": "system", "content": "Eres un asistente útil y amigable."}]

    # Añadir el nuevo mensaje del usuario
    messages.append({"role": "user", "content": previous_msgs_str+message})

    # Solicitud a OpenAI para generar la respuesta
    response = openai.ChatCompletion.create(
        engine="gpt-4-turbo",  # Nombre del deployment
        messages=messages,
        max_tokens=300,
        temperature=0.7
    )

    # Obtener la respuesta del asistente
    bot_response = response['choices'][0]['message']['content']

    # Devolver solo la respuesta del bot (sin historial)
    return bot_response

def chatbot_memory_last3(message, chat_history):
    # El mensaje se envía sin incluir historial previo

    previous_msgs_str = ""
    for msg in chat_history:
      previous_msgs_str += msg[0]
    messages = [{"role": "system", "content": "Eres un asistente útil y amigable."}]

    # Añadir el nuevo mensaje del usuario
    messages.append({"role": "user", "content": previous_msgs_str+message})

    # Solicitud a OpenAI para generar la respuesta
    response = openai.ChatCompletion.create(
        engine="gpt-4-turbo",  # Nombre del deployment
        messages=messages,
        max_tokens=300,
        temperature=0.7
    )

    # Obtener la respuesta del asistente
    bot_response = response['choices'][0]['message']['content']

    # Devolver solo la respuesta del bot (sin historial)
    return bot_response

def chatbot_memory_sumup(message, chat_history):
    # El mensaje se envía sin incluir historial previo

    previous_msgs_str = ""
    for msg in chat_history[-3:]:
      previous_msgs_str += msg[0]
    messages = [{"role": "system", "content": "Eres un asistente útil y amigable."}]

    messages.append({"role": "user", "content": "Resume los siguientes mensajes: "+previous_msgs_str})
    # Solicitud a OpenAI para generar la respuesta
    response_sumup = openai.ChatCompletion.create(
        engine="gpt-4-turbo",  # Nombre del deployment
        messages=messages,
        max_tokens=300,
        temperature=0.7
    )
    bot_response_sumup = response_sumup['choices'][0]['message']['content']

    messages = [{"role": "system", "content": "Eres un asistente útil y amigable."}]
    # Añadir el nuevo mensaje del usuario
    messages.append({"role": "user", "content": bot_response_sumup+message})
    print(bot_response_sumup)

    # Solicitud a OpenAI para generar la respuesta
    response = openai.ChatCompletion.create(
        engine="gpt-4-turbo",  # Nombre del deployment
        messages=messages,
        max_tokens=300,
        temperature=0.7
    )

    # Obtener la respuesta del asistente
    bot_response = response['choices'][0]['message']['content']

    # Devolver solo la respuesta del bot (sin historial)
    return bot_response

# Definir la interfaz de Gradio
def create_chat_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## Chatbot sin Memoria usando OpenAI")

        # Definir los componentes del chat
        chatbot = gr.Chatbot(label="Chat con OpenAI")
        msg = gr.Textbox(label="Tu mensaje")
        clear = gr.Button("Limpiar Chat")

        # Estado del historial de chat
        state = gr.State([])  # Estado para almacenar el historial del chat

        # Acción para enviar mensaje
        def send_message(user_message, chat_history):
            # Obtener la respuesta del chatbot sin utilizar historial
            # bot_message = chatbot_no_memory(user_message)
            #bot_message = chatbot_memory_full(user_message, chat_history)
            bot_message = chatbot_memory_sumup(user_message, chat_history)

            # Actualizar el historial de chat visible, pero sin afectar la memoria del bot
            chat_history.append((user_message, bot_message))

            # Devolver el mensaje del usuario y la respuesta acumulada en el historial
            return "", chat_history, chat_history

        # Conectar los componentes
        msg.submit(send_message, [msg, state], [msg, state, chatbot])
        clear.click(lambda: None, None, chatbot, queue=False)  # Limpiar el chat

    return demo

# Ejecución del chatbot

In [ ]:
# Ejecutar la interfaz
if __name__ == "__main__":
    demo = create_chat_interface()
    demo.launch(debug=True)